# Project 1: Linear Regression

a) Plain old linear regression, with no regularization. You must code this one by hand (i.e use equation 3.6 to find the betas).  Report the mean squared error on the test dataset. Replicate tables 3.1 and 3.2. You will not need the validation set for this part of the assignment.



b) Ridge regression. You must also code this one by hand(eq 3.44 to find the betas). Select the optimal value of Lambda by cross-validation using the validation dataset. Report the mean squared error on the test dataset, using the best lambda you found on the validation set. DO NOT USE THE TEST DATASET TO CHOOSE LAMBDA. Plot a ridge plot similar to figure 3.8, but you can just sweep the lambda parameter (you don't have to scale it to degrees of freedom).



c) Lasso regression: Use  the built in packages in sci-kit learn or MATLAB to do a Lasso regression. Select the optimal value of lambda as in part b) and also display a Lasso plot similar to figure 3.10, but again you can just sweep the lambda parameter. 



Next, download a dataset suitable for linear regression from UCI or another repository. For now, this should be a dataset that only has numerical features, with no missing values. Repeat the analysis above on this dataset.

In [72]:
import numpy as np
import matplotlib as plt
import pandas as pd
# import seaborn as sns
from scipy import stats


In [119]:
prostate_df = pd.read_csv('prostate.txt', delimiter='\t', index_col=0)
df_train = prostate_df.pop('train')
df_y = prostate_df.pop('lpsa')

In [120]:
# normalize data!
prostate_df = prostate_df.apply(stats.zscore)

In [122]:
train_df = prostate_df.loc[df_train == 'T']
val_df  = prostate_df.loc[df_train == 'F'][:15]
test_df  = prostate_df.loc[df_train == 'F'][15:]

print(len(train_df), len(val_df), len(test_df))

67 15 15


In [123]:
train_df

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
1,-1.645861,-2.016634,-1.872101,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957
2,-1.999313,-0.725759,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957
3,-1.587021,-2.200154,1.368234,-1.030029,-0.525657,-0.867655,0.344407,-0.156155
4,-2.178174,-0.812191,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957
5,-0.510513,-0.461218,-0.251933,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957
...,...,...,...,...,...,...,...,...
91,1.617422,1.109520,0.558151,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957
92,1.008835,0.114086,-0.386947,0.864484,1.902379,-0.867655,0.344407,-0.334356
93,1.262444,0.580608,0.558151,-1.030029,1.902379,1.079149,0.344407,1.269449
94,2.107397,0.628738,-2.682185,-1.030029,1.902379,1.688267,0.344407,0.556647


In [130]:
X_train = prostate_df.loc[df_train == 'T'] # shape: (67,8) -> (N, p)
y_train = df_y[df_train == 'T'] # shape: (67,)

X_val = prostate_df.loc[df_train == 'F'][:15] # shape: (67,8) -> (N, p)
y_val = df_y[df_train == 'F'][:15] # shape: (67,)

X_test = prostate_df.loc[df_train == 'F'][15:] # shape: (67,8) -> (N, p)
y_test = df_y[df_train == 'F'][15:] # shape: (67,)

# 1. Linear Regression

## Estimate Betas

In [137]:
X_train_b = np.hstack((np.ones((X_train.shape[0], 1)), X_train))    # add bias terms (ones) to first col -> (N, p+1)
X_test_b = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

beta = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train # shape: (9,), estimated coefficients

y_pred = np.dot(X_test_b, beta) # estimate for tests
mse = np.mean((y_test - y_pred)**2) # compute MSE

print("MSE on test set:", mse)

MSE on test set: 0.7776408048062767


## Plot Feature Correlation + Z-Score

In [133]:
X_train.corr()  # does it have to be a lower triangle shape?


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000


In [135]:
# get residuals
y_pred_train = X_train_b @ beta # estimate for trains
residuals = y_train - y_pred_train

# estimate var
var = np.sum((y_pred_train - y_train)**2) / (X_train_b.shape[0] - X_train_b.shape[1])

# std error
standard_errors = np.sqrt(np.diag(var * np.linalg.inv(X_train_b.T @ X_train_b)))    # sqrt term in z-score

# z-scores
z_scores = beta / standard_errors

In [136]:
terms = ['Intercept'] + [i for i in X_train.columns]

print(f"{'Terms':>15} {'Coefficient':>18} {'Std.Error':>15} {'Z-Score':>15}")
for i in range(len(beta)):
    print(f" {terms[i]:>15} {beta[i]:>15.2f} {standard_errors[i]:>15.2f} {z_scores[i]:>15.2f}")

          Terms        Coefficient       Std.Error         Z-Score
       Intercept            2.46            0.09           27.60
          lcavol            0.68            0.13            5.37
         lweight            0.26            0.10            2.75
             age           -0.14            0.10           -1.40
            lbph            0.21            0.10            2.06
             svi            0.30            0.12            2.47
             lcp           -0.29            0.15           -1.87
         gleason           -0.02            0.14           -0.15
           pgg45            0.27            0.15            1.74


# 2. Lasso Regression